In [33]:
import cv2
import numpy as np
import os 
import copy
from skimage.feature import hog

In [34]:
def non_max_suppression(boxes, overlap_thresh):
    # If there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # Initialize the list of picked indices
    picked_boxes = []

    # Grab the coordinates of the bounding boxes
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]

    # Compute the area of the bounding boxes and sort the bounding boxes by bottom-right y-coordinate
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = scores.argsort()[::-1]

    # Keep looping while some indexes still remain in the indexes list
    while len(idxs) > 0:
        # Grab the last index in the indexes list and add the index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        picked_boxes.append(boxes[i])
        # Find the largest (x, y) coordinates for the start of the bounding box and the smallest (x, y) coordinates for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # Compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # Compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # Delete all indexes from the index list that have overlap greater than the provided overlap threshold
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlap_thresh)[0])))
    # Return only the bounding boxes that were picked
    return picked_boxes


In [35]:
def run_single_scale_detector(classifier, image_path, scale_factor):
        detections = []
        # Load the test image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize the image to a fixed size
        resized_image = cv2.resize(gray, (100, 100))

        # Detect faces using the classifier
        faces = classifier.detectMultiScale(resized_image, scaleFactor=scale_factor, minNeighbors=5)

        # Convert the coordinates to original image scale
        for (x, y, w, h) in faces:
            x *= (image.shape[1] / 100)
            y *= (image.shape[0] / 100)
            w *= (image.shape[1] / 100)
            h *= (image.shape[0] / 100)
            detections.append((x, y, w, h))

        return detections

In [45]:
import cv2
import numpy as np

def sliding_window(image, step_size, window_size):
    # Slide a window across the image
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def detect_faces(image_path, step_size=20, window_size=(100, 100), overlap_thresh=0.3, target_size=(100, 100)):
    """
    Detects faces in the given image using sliding window and non-maximum suppression techniques.

    Parameters:
    image_path (str): Path to the image file.
    step_size (int): Step size for sliding window (default is 20).
    window_size (tuple): Size of the sliding window (default is (100, 100)).
    overlap_thresh (float): Overlap threshold for non-maximum suppression (default is 0.3).
    target_size (tuple): Target size for resizing the detected face (default is (100, 100)).

    Returns:
    face_encodings (list): Encoding of the detected face.
    """
    classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize list to store detected faces
    detections = []

    # Loop over the sliding window
    for (x, y, window) in sliding_window(gray, step_size, window_size):
        # Clone the original image and get the HOG face detector
        clone = gray.copy()
        patch = clone[y:y + window_size[1], x:x + window_size[0]]

        # Detect faces in the window
        faces = classifier.detectMultiScale(patch, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Add the bounding boxes of detected faces to the list
        for (x1, y1, w, h) in faces:
            detections.append((x + x1, y + y1, x + x1 + w, y + y1 + h))

    # If no faces are detected, return None
    if len(detections) == 0:
        return None

    # Select the best bounding box (currently the first one)
    (x1, y1, x2, y2) = detections[0]

    # Extract the face region
    face = image[y1:y2, x1:x2]

    # Resize the detected face to the target size
    face_resized = cv2.resize(face, target_size)

    # Return the detected face
    return face_resized


In [37]:
face = detect_faces('school/database_images/0001_c1_4.png', step_size=20, overlap_thresh=0.3)

In [38]:
print(face)

None


In [46]:
def create_database(directory):
    """
    Creates and stores encodings of detected faces from images in the specified directory.

    Parameters:
    directory (str): Path to the directory containing images.
    """
    face_encodings = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            face = detect_faces(image_path, window_size=(100, 100))
            if face is not None:
                face_encodings.append(face)
                labels.append(filename[:4])

    return face_encodings, labels
    


In [47]:
def search_face(face_encodings, query_encoding, labels, top_n=10):
    """
    Compares the query face encoding with pre-loaded encodings to identify similar faces.

    Parameters:
    face_encodings (list): List of pre-loaded face encodings.
    query_encoding (numpy.ndarray): Encoding of the query face.
    top_n (int): Number of top matches to return (default is 10).

    Returns:
    sorted_results (list): Sorted list of top matched faces.
    """
    distances = [np.linalg.norm(query_encoding - face) for face in face_encodings]
    sorted_indices = np.argsort(distances)[:top_n]
    sorted_results = [(face_encodings[i], distances[i], labels[i]) for i in sorted_indices]

    return sorted_results


In [48]:
directory = 'school/database_images'
face_encodings, labels = create_database(directory)

In [49]:
print(labels)

['0003', '0003', '0019', '0021', '0026', '0041', '0049', '0050', '0063', '0066', '0081', '0096', '0098', '0100', '0100', '0111', '0113', '0125', '0131', '0144', '0152', '0153']


In [50]:
print(len(labels))

22


In [51]:
predicted_labels = []
for filename in os.listdir('school/query/'):
  query_encoding = detect_faces(os.path.join('school/query/', filename))
  if query_encoding is None:
    predicted_labels.append('None')
    continue
  results = search_face(face_encodings, query_encoding, labels)
  for face, distance, label in results:
    predicted_labels.append(label)
    break

In [52]:
print(predicted_labels)

['None', 'None', '0153', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '0050', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '0050', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '0049', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',

In [53]:
# Function to compute rank-1 accuracy
def compute_rank1_accuracy(true_labels, predicted_labels):
    correct = 0
    total = len(true_labels)
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        if true_label == predicted_label[0]:
            correct += 1
    rank1_accuracy = correct / total
    return rank1_accuracy

# Sample true labels and predicted labels
true_labels = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015',
               '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030',
               '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045',
               '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060',
               '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075',
               '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0089', '0090',
               '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105',
               '0107', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120',
               '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0132', '0133', '0134', '0135',
               '0136', '0137', '0138', '0139', '0140', '0141', '0142', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150',
               '0151', '0152', '0153', '0154', '0155', '0156', '0157', '0158', '0159']


# Compute rank-1 accuracy
rank1_accuracy = compute_rank1_accuracy(true_labels, predicted_labels)
print("Rank-1 accuracy:", rank1_accuracy)


Rank-1 accuracy: 0.0


In [ ]:
# Function to compute rank-10 accuracy
def compute_rank10_accuracy(true_labels, predicted_labels):
    correct = 0
    total = len(true_labels)
    for true_label, predicted_label_list in zip(true_labels, predicted_labels):
        if true_label in predicted_label_list[:10]:
            correct += 1
    rank10_accuracy = correct / total
    return rank10_accuracy

# Sample true labels and predicted labels (where predicted labels are lists of top 10 identities)
true_labels = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015',
               '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030',
               '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045',
               '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059', '0060',
               '0061', '0062', '0063', '0064', '0065', '0066', '0067', '0068', '0069', '0070', '0071', '0072', '0073', '0074', '0075',
               '0076', '0077', '0078', '0079', '0080', '0081', '0082', '0083', '0084', '0085', '0086', '0087', '0089', '0090',
               '0091', '0092', '0093', '0094', '0095', '0096', '0097', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105',
               '0107', '0109', '0110', '0111', '0112', '0113', '0114', '0115', '0116', '0117', '0118', '0119', '0120',
               '0121', '0122', '0123', '0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0132', '0133', '0134', '0135',
               '0136', '0137', '0138', '0139', '0140', '0141', '0142', '0143', '0144', '0145', '0146', '0147', '0148', '0149', '0150',
               '0151', '0152', '0153', '0154', '0155', '0156', '0157', '0158', '0159']


# Compute rank-10 accuracy
rank10_accuracy = compute_rank10_accuracy(true_labels, predicted_labels)
print("Rank-10 accuracy:", rank10_accuracy)

Rank-10 accuracy: 0.0


In [ ]:
print(len(face_encodings))

2


In [59]:
img2 = cv2.imread('school/database_images/0001_c1_4.png')
# Converting image to grayscale 
gray_img = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 
  
# Loading the required haar-cascade xml classifier file 
haar_cascade = cv2.CascadeClassifier('Haarcascade_frontalface_default.xml') 
  
# Applying the face detection method on the grayscale image 
faces_rect = haar_cascade.detectMultiScale(gray_img, 1.1, 9) 
  
# Iterating through rectangles of detected faces 
for (x, y, w, h) in faces_rect: 
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2) 
  
cv2.imshow('Detected faces', img) 
  
cv2.waitKey(0) 

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
